<a href="https://colab.research.google.com/github/park-geun-hyeong/practice_pytorch/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !apt install unzip
# !unzip hw5_doghole_keeper.zip

In [14]:
import os
import gc
PATH = '/content/drive/MyDrive/PyTorch/homework/IT_HW5/hw5_doghole_keeper/'
os.chdir(PATH)

In [29]:
import numpy as np
import cv2
import glob
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms as T
import torchvision.models as models
import sys
sys.setrecursionlimit(10000)

### 1. Data preparation

In [16]:
transform = T.Compose([T.Resize(256), T.RandomCrop(224), T.ToTensor()])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [30]:
class Dataset(Dataset):
    def __init__(self, path='train'):
        super(Dataset, self).__init__()

        self.img_path = []
        bo_path = glob.glob(path + '/bo/*.jpg')
        notbo_path = glob.glob(path + '/not_bo/*.jpg')
        
        self.img_path = bo_path + notbo_path
        self.label = [1] * len(bo_path) + [0]*len(notbo_path)

        
    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])
        img_pil = Image.fromarray(img) ## pil object(from array to img)

        self.img_tensor = transform(img_pil)      
        self.label_tensor = torch.tensor(self.label[index])

        return self.img_tensor.to(device), self.label_tensor.to(device)
        
    def __len__(self):
        return len(self.img_path)

In [31]:
training_dataset = Dataset('train')
training_loader = DataLoader(dataset=training_dataset, batch_size=8, shuffle=True)

In [32]:
validation_dataset = Dataset('valid')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=8, shuffle=False)

### 2. Constructing a convolutional neural network

In [42]:
vgg16 = models.vgg16(pretrained = True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [46]:
for param in vgg16.parameters():
    param.requires_grad = False ##  ` fc layer를 제외하고 나머지 pretrained_parameter들은 freezing

In [47]:
num_features = vgg16.classifier[0].in_features
vgg16.classifier = nn.Sequential(

    nn.Linear(num_features, 4096),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(1024, 128),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(128, 2)
)

In [48]:
vgg16 = vgg16.to(device)

### 3. Loss function and optimization method

In [49]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.0001) ## 일반적으로 기존 model을 학습할때 사용했던 lr의 0.1배를 사용하면 좋다,

### 4. Training of a neural network

In [50]:
for epoch in range(10):
    loss_val = 0
    for itr, data in enumerate(training_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = vgg16(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)

Epoch: 1   , Loss: 5.729405213147402
Epoch: 2   , Loss: 1.3371542356035206
Epoch: 3   , Loss: 1.3224139358244429
Epoch: 4   , Loss: 0.3523135763261962
Epoch: 5   , Loss: 0.07477253729681621
Epoch: 6   , Loss: 0.024201838707540446
Epoch: 7   , Loss: 0.011665237727058297
Epoch: 8   , Loss: 0.0070560451293877335
Epoch: 9   , Loss: 0.007153801831178974
Epoch: 10   , Loss: 0.003982487985716432


### 5. Prediction and evluation for the validation set

In [51]:
pred_list = []
label_list = []

for itr, data in enumerate(validation_loader):
    inputs, labels = data

    pred = vgg16(inputs)
    pred_category = torch.argmax(pred, dim=1)

    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list) == np.array(label_list) )
print("Validation accuracy:", accu)

Validation accuracy: 0.9333333333333333


In [52]:
pred_list

[tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0')]

In [53]:
np.array([i.detach().cpu() for i in pred_list])

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])